In [13]:
import altair as alt
import ibis_vega_transform

import warnings
try:
    from ibis.backends import omniscidb as ibis_omniscidb
except ImportError as msg:
    warnings.warn(str(msg))
    from ibis import omniscidb as ibis_omniscidb

conn = ibis_omniscidb.connect(
    host='bewdy.mapd.com', user='mapd', password='HyperInteractive',
    port=6274, database='mapd', protocol= 'binary'
)

In [14]:
t = conn.table('github')

In [15]:
t.head().execute()

,type,public_,repo_id,repo_name,repo_url,actor_id,actor_login,actor_gravatar_id,actor_avatar_url,actor_url,org_id,org_login,org_gravatar_id,org_avatar_url,org_url,created_at,id
0,PushEvent,True,17112912,dannyshmueli/shmueli-recipes,https://api.github.com/repos/dannyshmueli/shmu...,4062328,None,None,https://avatars.githubusercontent.com/u/4062328?,https://api.github.com/users/dannyshmueli,NaN,None,None,None,None,2015-01-01 12:02:20,2489583492
1,PushEvent,True,6058234,sakai-mirror/melete,https://api.github.com/repos/sakai-mirror/melete,2033581,None,None,https://avatars.githubusercontent.com/u/2033581?,https://api.github.com/users/mirror-updates,2033545.0,sakai-mirror,None,https://avatars.githubusercontent.com/u/2033545?,https://api.github.com/orgs/sakai-mirror,2015-01-01 00:39:36,2489387308
2,PushEvent,True,22869205,harada4atsushi/dotfiles,https://api.github.com/repos/harada4atsushi/do...,1818925,None,None,https://avatars.githubusercontent.com/u/1818925?,https://api.github.com/users/harada4atsushi,NaN,None,None,None,None,2015-01-01 11:11:08,2489567244
3,PushEvent,True,28208522,thiagopaz01/iOSTest,https://api.github.com/repos/thiagopaz01/iOSTest,8958527,None,None,https://avatars.githubusercontent.com/u/8958527?,https://api.github.com/users/thiagopaz01,NaN,None,None,None,None,2015-01-01 12:02:20,2489583496
4,PushEvent,True,6058234,sakai-mirror/melete,https://api.github.com/repos/sakai-mirror/melete,2033581,None,None,https://avatars.githubusercontent.com/u/2033581?,https://api.github.com/users/mirror-updates,2033545.0,sakai-mirror,None,https://avatars.githubusercontent.com/u/2033545?,https://api.github.com/orgs/sakai-mirror,2015-01-01 00:39:36,2489387309


In [16]:
over_time = alt.Chart(t).mark_line().encode(
    x='yearmonth(created_at):T',
    y='count()'
)

In [17]:
repo_dropdown = alt.binding_select(options=[
    'jupyterlab/jupyterlab',
    'vega/vega',
    'altair-viz/altair',
    'omnisci/omniscidb',
    'mapd/mapd-core'
])
repo_selection = alt.selection_single(fields=['repo_name'], bind=repo_dropdown)

In [18]:
over_time_selected = over_time.add_selection(
    repo_selection
).transform_filter(
    repo_selection
)

In [19]:
over_time_selected

alt.Chart(...)

In [20]:
types = alt.Chart(t).mark_bar().encode(
    x='count()',
    y=alt.Y('type:O', sort=alt.Sort(encoding='x', order='descending'))
).add_selection(
    repo_selection
).transform_filter(
    repo_selection
)
types

alt.Chart(...)

In [21]:
time_selection = alt.selection_interval(
    fields=['created_at'],
    encodings=['x']
)

In [22]:
(over_time_selected.add_selection(
    time_selection
) & (types.transform_filter(
    time_selection
).properties(height=300))).properties(
    padding=150
)

alt.VConcatChart(...)